# **<font color='green'>TRABAJO FINAL</font>**

## <font color='blue'> Uniprot </font> 
- Para este trabajo se va realizar web scraping a la pagina UniProt, dicha pagina tiene como funcion proporcionar a la comunidad científica un recurso completo, de alta calidad y de libre acceso de secuencia de proteínas e información funcional.

In [1]:
#importando librerias 
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [2]:
# Lee todas las páginas que hay en el excel, obvia la primera fila y además solo usa las primeras 18 columnas
df= pd.read_excel('02 Ch Genes ordenados con vecinos.xlsx',engine='openpyxl',sheet_name=None,skiprows=[0],usecols=range(18))

In [3]:
# Se extrae los nombres de las páginas del excel y se guarda en el objeto "nombres"
nombres=list(df.keys())
nombres

['Información',
 'C1 Milagros',
 'C2 Wilmer',
 'C3 Gisell',
 'C4 Jerica',
 'C5 Manuel',
 'C6 Jerica',
 'C7 Mariana',
 'C8 Rosha',
 'C9 Jerica',
 'C10 Lucero',
 'C11 Nicole',
 'C12 Gisell',
 'C13 Lucero',
 'C14 Wilmer',
 'C15Milagros',
 'C16',
 'C17',
 'C18',
 'Hoja 20',
 'Hoja 21',
 'Hoja 22',
 'Hoja 23']

In [4]:
hojas=[]               # Se crea una lista vacia y se almacena en el objeto "hojas"
for nombre in nombres: # Se crea un bucle para cada nombre de las páginas en excel en la lista de "nombres"
    if 'Gen Abrev' in list(df.get(nombre).columns): # Si es que en la página de excel esta la columna "Gen Abrev", se continua con la iteración
        hojas.append(df.get(nombre)[df.get(nombre)['Gen Abrev'].notnull()]) # Agregar los datos de cada página en la lista de "hojas"

In [5]:
genes=[]           # Se crea una lista vacia y se almacena en el objeto "genes"
for hoja in hojas: # Se crea un bucle para cada página en la lista de hojas
    genes.append(hoja['Gen Abrev'].tolist()) # Se guardan los datos de la columna "Gen Abrev" en la lista de "genes"

In [6]:
#vista de los 6 primeros genes de la primera hoja del excel
genes[0][:6]

['TRNAS-GGA',
 'LOC102173449',
 'LOC102169333',
 'LOC108636757',
 'LOC102187712',
 'VGLL3']

In [7]:
links=[]              # Se crea una lista vacia y se almacena en el objeto "links"
for lista in genes:   # Se crea un bucle para cada lista de genes en cada pagina
    links1=[]         # Se crea una lista vacia y se almancena en el objeto "links1"
    for gen in lista: # Se crea un bucle para cada gen en cada lista de genes
        if type(gen)==str:  # Si el objeto gen es de tipo string, se continua con la iteración
            url = "https://www.uniprot.org/uniprot/?query="+gen+"+Capra+Hircus&sort=score" # Se crea el objeto "url" para cada gen
            #print(url)
            links1.append(url) # Se guardan todos los url en la lista "links1"
    links.append(links1)

In [8]:
# Se crea una función para determinar el número de genes que se desea extraer del excel
def num_genes():
    #se pregunta si se desea un número exacto o todos los genes
    x=input("¿Desea extraer un número exacto de genes?, S/N")
    #si la respuesta es si
    if x=='S':
        #se ingresa la cantidad de datos que se solicita
        Ngenes=int(input("Ingrese el número de genes que desea extraer de cada hoja: "))
        return Ngenes
    #si la respuesta es no
    elif x=='N':
        # se retorna un número muy grande que sobrepase la cantidad de genes en las hojas 
        return 99999999
    # en caso de colocar una respuesta erronea se vuelve a la pregunta inicial
    else:
        return num_genes()

### <font color='blue'> Descripcion de la libreria tqdm </font> 
- Es un pequeño módulo que permite crear una barra de progreso basada en texto, que es desplegada en pantalla a partir de un bucle. Además de la barra de progreso, incluye porcentaje, tiempo transcurrido y remanente, y el número de iteraciones por segundo.

In [13]:
from tqdm.notebook import tqdm    # Se importa la libreria tqdm.notebook para una mejor visualización en el proceso de extracción de los datos
writer = pd.ExcelWriter('GenesRepository.xlsx', engine='openpyxl') 
n_genes=num_genes()               # Se guarda la función en un objeto por comodidad
print("Progreso de las hojas del excel")
for j in tqdm(range(len(links))): # Se va de hoja en hoja 
    lista = links[j]
    htmlList=[]
    datos=[]
    print("\nProgreso de los genes en la hoja actual")
    #Se solicita el mínimo de entre la cantidad de elementos de la lista y la cantidad de genes solicitados 
    for i in tqdm(range(min(len(lista),n_genes))):#Se va de gen en gen 
        print("\rHoja Actual: "+nombres[j+1]+"     Gen actual: "+genes[j][i],end="")
        link=lista[i]
        r = requests.get(link) # Acceder a los datos de los links
        soup = BeautifulSoup(r.text, 'lxml')
        html=soup
        datos1=[]
        if html.find("td",{"class":"entryID"})!=None:
            datos1.append(html.find("td",{'class':"entryID"}).a.text)
            # Añadimos a la lista datos1 los valores que tengan como etiqueda td y  clase EntryID, el cual nos va permitir extraer los 
            # Entryname del documento, con el atributo find_next_sibiling atraemos la columna que tenga td y la convertimos en texto 
            datos1.append(html.find("td",{'class':"entryID"}).find_next_sibling("td").text)
            # Creamos un objeto y le añadimos el link de la pagina y concatenamos los valores obtenidos sobre entryID con el fin 
            # de vaya variando para todos los genes 
            link='https://www.uniprot.org'+html.find("td",{'class':"entryID"}).a['href']
            # Volvemos hacer un request, pero esta vez para link 
            r1=requests.get(link)
            #Ahora creamos un objeto y vamos abrir el archivo con Beatifulsoup con el parser lxml
            #Lo cual va lograr que el proceso sea mas largo.
            soup1=BeautifulSoup(r1.text, 'lxml')
            #Extraemos y añadimos a la lista datos 1, con el metodo find extraemos los valores 
            #Que tengan etiqueta div y que sea de propiedad id y valor content-protein, y que la etiqueta sea h1 en este caso seran los nombres de las proteinas. 
            datos1.append(soup1.find("div",{'id':"content-protein"}).h1.text)
            # Ahora creamos un bucle, si el valor que buscamos con etiqueta div y que sea de propiedad id y valor content-gene es diferente de nulo
            # Se continua con el proceso, es decir si el nombre del gen es diferente de Nulo ejecutar las siguientes lineas. 
            if soup1.find("div",{'id':"content-gene"}).h2!=None:
                #Mediante este codigo buscamos y añadimos el valor que tenga etiqueta div y que sea de propiedad id y valor content-gene, tambien debe cumplir con tener la etiqueta h2
                datos1.append(soup1.find("div",{'id':"content-gene"}).h2.text)
            #De lo contrario 
            else:
                # En caso de no encontrar nada, adicionar un valor nulo a la lista  
                datos1.append(None)
            # Buscamos y  añadimos el organismo del gen, estos deben cumplir la condicion que tenga etiqueta div y que sea de propiedad id y valor conten-organism y 
            #con el metodo split separamos el nombre en dos y con cero indicamos que queremos añadir solo el primer elemento de la lista 
            datos1.append(soup1.find("div",{'id':"content-organism"}).text.split(" (")[0])
            #Buscamos y  añadimos el estatus del gen, estos deben cumplir la condicion que tenga etiqueta div y que sea de propiedad id y valor conten-status, ademas debe cumplir con tener la etiqueta a para finalmente extraer el texto 
            datos1.append(soup1.find("div",{'id':"content-status"}).a.text)
            #Para poder obtener los procesos biologicos, se crea un funcion la cual nos va permitir encontrar en los html oen los sub
            # la propiedad name h4, luego se realizara un tag.text y encontrara "GO - Biological process"
            tag = soup1.find(lambda tag:tag.name=="h4" and "GO - Biological process" in tag.text)
            #Se crea un bucle, si en caso de no encontrar el valor el tag 
            if(tag==None):
                # Se debe adicionar a la lista un valor nulo 
                datos1.append(None)
            #De lo contraria, si existen valores para tag 
            else: 
                # Como queremos el contenido de procesos biologicos, eso lo vamos a lograr con el metodo next_sibling  para despues almacernarlo en una lista 
                lists= tag.next_sibling
                # Creamos una lista vacia
                listaTemp=[]
                # Creanos un bucle for 
                for l in lists:
                    #Añadimos a la lista los contenidos, pero como salen acompañados de la palabra "Souce" la separamos mediante el metodo split, para asi con el 0 seleccionar solo el primer elemento de la lista
                    listaTemp.append(l.text.split(" Source")[0])
                #Esto sirve para que en la data frame que vamos a trabajar no aparezca como lista, sino los valores aparezcan separados por comas 
                datos1.append(",".join(listaTemp))
        if not any(datos1):# si no existe ningun dato en la lista "datos1"
            datos1=[None,None,None,None,None,None,None]        
        #Añadimos los valores de la lista datos1
        datos.append(datos1)
    #Creamos un data frame y lo guardamos en el objeto df_datos, ademas indicamos el nombre de sus columnas 
    df_datos=pd.DataFrame(datos, columns=["Entry","Entry name","Protein","Gene","Organism","Status","GO - Biological process"])
    # Creamos un data frame para los genes y una columna con el nombre de Gen Abrev 
    df_genes=pd.DataFrame(genes[j],columns=["Gen Abrev"])
    #Concatenamos ambos data frame y con axis decimos que sea mediante columnas
    df_final=pd.concat([df_genes.head(n_genes),df_datos],axis=1)
    #Se coloca un título al index
    df_final.index.name='N°'
    #Ahora se cambia los valores, si dentro aparece "None" se cambia por "NaN"
    df_final=df_final.fillna(value='NaN')
    #Con esta parte se escribe en la hoja de excel y asi va yendo de una en una. 
    df_final.to_excel(writer,sheet_name=nombres[j+1])
    #Con el metodo .save guardamos toda la informacion para que sea visible 
    writer.save()# Se cierra el simulador de webDriver 

¿Desea extraer un número exacto de genes?, S/NS
Ingrese el número de genes que desea extraer de cada hoja: 2
Progreso de las hojas del excel


  0%|          | 0/18 [00:00<?, ?it/s]


Progreso de los genes en la hoja actual


  0%|          | 0/2 [00:00<?, ?it/s]

Hoja Actual: C1 Milagros     Gen actual: LOC102173449
Progreso de los genes en la hoja actual


...

Hoja Actual: C16     Gen actual: BECN2
Progreso de los genes en la hoja actual


  0%|          | 0/2 [00:00<?, ?it/s]

Hoja Actual: C17     Gen actual: LOC102173595
Progreso de los genes en la hoja actual


  0%|          | 0/2 [00:00<?, ?it/s]

Hoja Actual: C18     Gen actual: LOC106503091